In [96]:
%run ./../DataFiles_and_Notebooks/00_AdvancedPythonConcepts/talktools.py

# Making Python Faster

<img src="slides/warp1.jpg">


<h2> ... and using legacy code</h2>

- we've already seen `numexpr`, parallelization, etc.

- Python is _slow_ ... it's interpreted on the fly

- no static typing ... even integers are objects (bulky memory!)

- what if we want to write Python, but use it as a *glue* to fast C-code?

<pre>
Premature optimization is the root of all evil 
   -- Donald Knuth
</pre>

<pre>
C is premature optimization
   -- Kanye West
</pre>


## Profiling ##

You already know that Python comes with batteries include, and performance profiling is no exception.

You can keep track of how much time each *function* is taking up using tools from the standard library.

Here's the [documentation of `profile` and `cProfile`](http://docs.python.org/3/library/profile.html), but you probably won't need to use them directly.  A profile is a set of statistics that describes how often and for how long various parts of the program executed. These statistics can be formatted into reports via the pstats module.

In [ ]:
import cProfile
import re
cProfile.run('re.compile("fizz|buzz")') # run a piece of code

In [ ]:
import cProfile, pstats, io
pr = cProfile.Profile()
pr.enable()

# here's the code you want to profile
def waste_of_time(n=1000):
    [x for x in range(n)]
[waste_of_time(y) for y in range(10000)]
## end of code you want to profile

pr.disable()
s = io.StringIO()
sortby = 'cumulative'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

## There's a magic for that!

### `%timeit` to learn how long it takes a chunk of code to run

### `%prun` for function-by-function breakdown of code in your namespace

### `%run -p` for function-by-function breakdown of running a whole file

In [ ]:
cd demos/profile

In [ ]:
!cat sometask.py

In [ ]:
import sometask

In [ ]:
%prun sometask.execute()

In [ ]:
%run -p sometask.py

With the `-D` flag, you can dump the profile to binary file that external tools can use.


You  can also produce this .profile without Jupyter using:

```bash
    python -m cProfile -o sometask.profile sometask.py
```

In [ ]:
%run -p -D sometask.profile sometask.py

In [ ]:
!head sometask.profile

In [ ]:
!pip install snakeviz

In [ ]:
!snakeviz sometask.profile

In [ ]:
%load_ext snakeviz

In [ ]:
%%snakeviz
sometask.execute()

## Profiling line-by-line

    %lprun
    
This magic is not built into Jupyter, it is provided by the [line_profiler package by Robert Kern](http://pythonhosted.org/line_profiler/).

In [ ]:
!conda install line_profiler -y

In [ ]:
%load_ext line_profiler

Run the code, but only run the line profiler on the function `square()`

In [ ]:
import sometask

In [ ]:
!cat sometask.py

In [ ]:
%lprun -f sometask.square sometask.execute()

## Profiling memory usage (line-by-line)

    %mprun
    
        
This magic is not built into Jupyter, it is provided by the [memory_profiler package by Fabian Pedregosa](https://pypi.python.org/pypi/memory_profiler).

In [ ]:
!pip install  -U memory_profiler

In [ ]:
%load_ext memory_profiler

In [ ]:
%mprun?

In [ ]:
cd demos/profile/

In [ ]:
import sometask

In [ ]:
!cat sometask.py

In [ ]:
%mprun -f sometask.square sometask.execute()

# Other tools for profiling:

[gprof](http://en.wikipedia.org/wiki/Gprof) -- command line profiling tool for C code. [GNU Gprof documentation](https://sourceware.org/binutils/docs/gprof/) is pretty good.

[valgrind](http://valgrind.org/docs/manual/cl-manual.html) -- very complex suite for analyzing callgrind and kcachegrind. "*Valgrind is an instrumentation framework for building dynamic analysis tools. There are Valgrind tools that can automatically detect many memory management and threading bugs, and profile your programs in detail.*"

[Timing and Profiling in IPython](http://pynash.org/2013/03/06/timing-and-profiling/) : Blogpost at the PyNash

Calculate the variance

$$\sigma^2 = \frac {\sum_{i=1}^N (x_i - \sum_{j=1}^N x_j/N)^2}{N - 1}. $$

problem here is that you must first pass over all the data (to get the mean) then pass over the data again.  There's a bunch of Pythonic ways to do this. Here's a few...

In [ ]:
#%%writefile var.py
def variance(data):
    sample_mean = 0.0
    
    # 1st loop
    for x in data:
        sample_mean += float(x)
    
    sample_mean /= len(data)
    
    # second loop
    sum_of_squared_errors = 0.0
    for x in data:
        sum_of_squared_errors += (float(x) - sample_mean) ** 2
    
    return sum_of_squared_errors / (float(len(data)) - 1.0)
    
def variance0(data):
    sample_mean = sum(data) / len(data) # loop 1
    sum_of_squared_errors = sum((i - sample_mean) ** 2 for i in data) # loop 2
    return sum_of_squared_errors / (len(data) - 1)

import functools
def variance1(data):
    mean = float(functools.reduce(lambda x,y : x+y, data)) / len(data)
    return functools.reduce(lambda x,y: x+y, map(lambda xi: (xi-mean)**2, data))/ (len(data) - 1)

def execute():
    variance(range(100000))
    
def execute1():
    variance1(range(100000))

In [ ]:
%timeit variance(range(100000))

In [ ]:
%timeit variance0(range(100000))

In [ ]:
%timeit variance1(range(100000))

We'd like to do this with just one pass over the data. Have a look at Welford's Method (1962):

In [ ]:
def online_variance(data):
    mean,M2= 0.,0.
    for n,d in enumerate(data):
        delta = d - mean
        mean += delta/(n + 1)
        M2 += delta*(d - mean)  
    return M2/n

In [ ]:
%timeit online_variance(range(100000))

In [ ]:
import var
import importlib
importlib.reload(var)

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f var.variance1 var.execute1()

## Numba  ##

LLVM compiler for python (brought to you by Continuum). In short: Python code can be converted to highly efficient compiled code in real-time. 

Should be on everyone's conda installation already. But you might want to update (it's been changed recently):

   `conda update numba`

Docs: http://numba.pydata.org/numba-doc/0.29.0/index.html

In [ ]:
!conda install numba -y

In [ ]:
import numba
numba.__version__

How compilers usually work:

<img src="http://www.aosabook.org/images/llvm/SimpleCompiler.png">

How LLVM works:

<img src="http://www.aosabook.org/images/llvm/RetargetableCompiler.png">

Credit: http://www.aosabook.org/en/llvm.html

(Via http://nbviewer.ipython.org/urls/raw.github.com/aterrel/HPCPythonSC2012/master/02_Speeding_Python.ipynb)

In [ ]:
from numba import int32, float32
print(int32[:])           # 1D int32 array
print(float32[:,:])       # 2D float32 array
print(int32[:,:,:,:])     # 4D int32 array

Function types are created from "calling" a Numba type object.

In [ ]:
from numba import void, int32, float32, complex64
print(complex64(int32, float32, complex64))
print(float32())                             # no arguments
print(void(float32))                         # return nothing
print(void(float32[:], int32[:]))

The `@jit` decorator returns a compiled version of the function using the input types and the output types of the function. You can specify the type using out_type(in_type, ...) syntax. Array inputs can be specified using `[:,:]` appended to the type.

In [ ]:
from numba import jit, int32, float32, complex64, autojit

@jit(complex64(int32, float32, complex64))
def bar(a, b, c):
    return a + b  * c

@jit(complex64(int32, float32, complex64))
def foo(a, b, c):
    return a + b  * c

print(foo)
print(foo(1, 2.0, 3.0j))

In [ ]:
import math
a = math.sqrt((1+2j).real**2 + (1+2j).imag**2)

foo(2,a,0j)

In [ ]:
import math

@jit
def square(x):
    return x ** 2

@jit
def hypot(x, y):
    return math.sqrt(square(x) + square(y))

In [ ]:
hypot(4,5)

The `@autojit` decorator does not require you to specify any types. It watches for what types you call the function with and infers the type of the return. If there is a previously compiled version of the code available it uses it, if not it generates machine code for the function and then executes that code.

In [ ]:
@autojit
def f2(x,y):
    return x + y

print(f2("a","b"))
print(f2(1,1+0j))

In [ ]:
from numba import autojit

@autojit
def n_variance(data):
    sample_mean = 0.0
    
    # 1st loop
    for x in data:
        sample_mean =  sample_mean + float(x)
    
    sample_mean = sample_mean / len(data)
    
    # second loop
    sum_of_squared_errors = 0.0
    for x in data:
        sum_of_squared_errors += (float(x) - sample_mean) ** 2
    
    ret =  sum_of_squared_errors / (float(len(data)) - 1.0)
    return ret

In [ ]:
%timeit n_variance(list(range(100000)))

In [ ]:
n_ovariance = autojit(online_variance)

In [ ]:
%timeit n_ovariance(list(range(100000)))

List comprehension and other fancy Python tricks are not supported yet in numba

In [ ]:
n_variance0 = autojit(variance0)
n_variance0(list(range(1000)))

In [ ]:
import numpy

def filter2d(image, filt):
    M, N = image.shape
    Mf, Nf = filt.shape
    Mf2 = Mf // 2
    Nf2 = Nf // 2
    result = numpy.zeros_like(image)
    for i in range(Mf2, M - Mf2):
        for j in range(Nf2, N - Nf2):
            num = 0.0
            for ii in range(Mf):
                for jj in range(Nf):
                    num += (filt[Mf-1-ii, Nf-1-jj] * image[i-Mf2+ii, j-Nf2+jj])
            result[i, j] = num
    return result

That kind of quadruply-nested for-loop is going to be quite slow.
Using Numba we can compile this code to LLVM which then gets
compiled to machine code:

In [ ]:
from numba import double, jit

numbafilter_2d = jit(double[:,:](double[:,:], double[:,:]))(filter2d)

# Now fastfilter_2d runs at speeds as if you had first translated
# it to C, compiled the code and wrapped it with Python
image = numpy.random.random((100, 100))
filt = numpy.random.random((10, 10))
res = numbafilter_2d(image, filt)

In [ ]:
%timeit numbafilter_2d(image, filt)

In [ ]:
%timeit filter2d(image, filt)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(res)

In [ ]:
plt.imshow(image)

Numba can also compile for GPUs. See http://numba.pydata.org/numba-doc/0.37.0/cuda/index.html

<img src="http://pypy.org/image/pypy-logo.png">

Python interpreter and C/JIT compiler. See http://pypy.org/

*“If you want your code to run faster, you should probably just use PyPy.”*
   — Guido van Rossum (creator of Python)


# Breakout

a. Write a nested for loop that does elementwize array multiplication (on `A` and `B`) and prints the results. 

b. Get the runtime speed for A.shape = (30,50) and B.shape = (50,15)

c. Try using numba to make it faster.

d. How does the numba speed compare to the native matrix multiplication in numpy (`numpy.dot`)?

## Ctypes ##

ctypes is a foreign function library for Python. It provides C compatible data types, and allows calling functions in DLLs or shared libraries. It can be used to wrap these libraries in pure Python.

https://docs.python.org/3/library/ctypes.html

In [ ]:
from ctypes import *

In [ ]:
p = create_string_buffer(b"Hello", 10)  # create a 10 byte buffer

In [ ]:
p

In [ ]:
p.value

In [ ]:
print(sizeof(p)) ; repr(p.raw)

In [ ]:
repr(p.raw)

In [ ]:
p.value = b"Bye."

In [ ]:
print(sizeof(p)) ; repr(p.raw)

<table border="1" class="docutils">
<colgroup>
<col width="24%" />
<col width="46%" />
<col width="30%" />
</colgroup>
<thead valign="bottom">
<tr><th class="head">ctypes type</th>
<th class="head">C type</th>
<th class="head">Python type</th>
</tr>
</thead>
<tbody valign="top">
<tr><td><a class="reference internal" href="#ctypes.c_bool" title="ctypes.c_bool"><tt class="xref py py-class docutils literal"><span class="pre">c_bool</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">_Bool</span></tt></td>
<td>bool (1)</td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_char" title="ctypes.c_char"><tt class="xref py py-class docutils literal"><span class="pre">c_char</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">char</span></tt></td>
<td>1-character string</td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_wchar" title="ctypes.c_wchar"><tt class="xref py py-class docutils literal"><span class="pre">c_wchar</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">wchar_t</span></tt></td>
<td>1-character unicode string</td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_byte" title="ctypes.c_byte"><tt class="xref py py-class docutils literal"><span class="pre">c_byte</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">char</span></tt></td>
<td>int/long</td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_ubyte" title="ctypes.c_ubyte"><tt class="xref py py-class docutils literal"><span class="pre">c_ubyte</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">unsigned</span> <span class="pre">char</span></tt></td>
<td>int/long</td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_short" title="ctypes.c_short"><tt class="xref py py-class docutils literal"><span class="pre">c_short</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">short</span></tt></td>
<td>int/long</td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_ushort" title="ctypes.c_ushort"><tt class="xref py py-class docutils literal"><span class="pre">c_ushort</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">unsigned</span> <span class="pre">short</span></tt></td>
<td>int/long</td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_int" title="ctypes.c_int"><tt class="xref py py-class docutils literal"><span class="pre">c_int</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">int</span></tt></td>
<td>int/long</td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_uint" title="ctypes.c_uint"><tt class="xref py py-class docutils literal"><span class="pre">c_uint</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">unsigned</span> <span class="pre">int</span></tt></td>
<td>int/long</td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_long" title="ctypes.c_long"><tt class="xref py py-class docutils literal"><span class="pre">c_long</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">long</span></tt></td>
<td>int/long</td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_ulong" title="ctypes.c_ulong"><tt class="xref py py-class docutils literal"><span class="pre">c_ulong</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">unsigned</span> <span class="pre">long</span></tt></td>
<td>int/long</td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_longlong" title="ctypes.c_longlong"><tt class="xref py py-class docutils literal"><span class="pre">c_longlong</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">__int64</span></tt> or <tt class="xref c c-type docutils literal"><span class="pre">long</span> <span class="pre">long</span></tt></td>
<td>int/long</td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_ulonglong" title="ctypes.c_ulonglong"><tt class="xref py py-class docutils literal"><span class="pre">c_ulonglong</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">unsigned</span> <span class="pre">__int64</span></tt> or
<tt class="xref c c-type docutils literal"><span class="pre">unsigned</span> <span class="pre">long</span> <span class="pre">long</span></tt></td>
<td>int/long</td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_float" title="ctypes.c_float"><tt class="xref py py-class docutils literal"><span class="pre">c_float</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">float</span></tt></td>
<td>float</td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_double" title="ctypes.c_double"><tt class="xref py py-class docutils literal"><span class="pre">c_double</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">double</span></tt></td>
<td>float</td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_longdouble" title="ctypes.c_longdouble"><tt class="xref py py-class docutils literal"><span class="pre">c_longdouble</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">long</span> <span class="pre">double</span></tt></td>
<td>float</td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_char_p" title="ctypes.c_char_p"><tt class="xref py py-class docutils literal"><span class="pre">c_char_p</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">char</span> <span class="pre">*</span></tt> (NUL terminated)</td>
<td>string or <tt class="xref docutils literal"><span class="pre">None</span></tt></td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_wchar_p" title="ctypes.c_wchar_p"><tt class="xref py py-class docutils literal"><span class="pre">c_wchar_p</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">wchar_t</span> <span class="pre">*</span></tt> (NUL terminated)</td>
<td>unicode or <tt class="xref docutils literal"><span class="pre">None</span></tt></td>
</tr>
<tr><td><a class="reference internal" href="#ctypes.c_void_p" title="ctypes.c_void_p"><tt class="xref py py-class docutils literal"><span class="pre">c_void_p</span></tt></a></td>
<td><tt class="xref c c-type docutils literal"><span class="pre">void</span> <span class="pre">*</span></tt></td>
<td>int/long or <tt class="xref docutils literal"><span class="pre">None</span></tt></td>
</tr>
</tbody>
</table>

In [ ]:
x = c_int(100)

In [ ]:
print(x)

In [ ]:
type(x.value)

In [ ]:
sizeof(x)

In [ ]:
y = 100

In [ ]:
import sys
sys.getsizeof(y)

In [ ]:
sys.getsizeof(x)

In [ ]:
%%file test.c
#include <stdio.h>

void myprint(void);

void myprint()
{
    printf("hello world\n");
}

In [ ]:
!cat test.c

In [ ]:
# linux
#!gcc -shared -Wl,-soname,test -o test.so -fPIC test.c

# or... for Mac OS X 
!gcc -shared -Wl,-install_name,testlib.so -o test.so -fPIC test.c

In [ ]:
!ls -lat |head

In [ ]:
%%file myctest.py
import ctypes
myctest = ctypes.CDLL('./test.so')
myctest.myprint()

In [ ]:
import ctypes
myctest = ctypes.CDLL('./test.so')
myctest.myprint()

In [ ]:
%%file test1.c
int add(int a, int b);

int add(int a, int b)
{
    return(a+b);
}

In [ ]:
!gcc -shared -Wl,-install_name,test1lib.so -o test1.so -fPIC test1.c

In [ ]:
from ctypes import *

# load the shared object
libtest = cdll.LoadLibrary('./test1.so')

# call the function, yes it is as simple as that!
print(libtest.add(100, 200))

In [ ]:
%timeit libtest.add(10,20)

In [ ]:
%timeit sum([10,20])

In [ ]:
sum?

In [ ]:
%timeit 10 + 20

For a whole lot of timing comparisons, see 
http://nbviewer.ipython.org/url/jakevdp.github.io/downloads/notebooks/NumbaCython.ipynb

## f2py 

<pre>
...automatically construct an extension module that interfaces to routines in Fortran 77/90/95 code. It has the ability to parse Fortran 77/90/95 code and automatically generate Python signatures for the subroutines it encounters, or you can guide how the subroutine interfaces with Python by constructing an interface-definition-file (or modifying the f2py-produced one).
</pre>

It's part of numpy!

(http://docs.scipy.org/doc/numpy/user/c-info.python-as-glue.html#f2py)

In [ ]:
%%file pairwise_fort.f

      subroutine pairwise_fort(X,D,m,n)
          integer :: n,m
          double precision, intent(in) :: X(m,n)
          double precision, intent(out) :: D(m,m) 
          integer :: i,j,k
          double precision :: r 
          do i = 1,m 
              do j = 1,m 
                  r = 0
                  do k = 1,n 
                      r = r + (X(i,k) - X(j,k)) * (X(i,k) - X(j,k)) 
                  end do 
                  D(i,j) = sqrt(r) 
              end do 
          end do 
      end subroutine pairwise_fort

In [ ]:
!f2py -c pairwise_fort.f -m pairwise_fort > /dev/null

In [ ]:
import numpy as np
X = np.random.random((1000, 3))
from pairwise_fort import pairwise_fort
XF = np.asarray(X, order='F')
%timeit pairwise_fort(XF)